In [1]:
# To remove deprecated warnings from the tensorflow
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
PATH = os.getcwd()

In [3]:
DATA_PATH = os.path.join(PATH, 'brain_tumor_dataset')
data_dir_list = os.listdir(DATA_PATH)

In [4]:
print(data_dir_list)

['no', 'yes']


In [5]:
import cv2

classes_names_list=[]
img_data_list=[]

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224, 224))
        (b, g, r)=cv2.split(input_img_resize) 
        img=cv2.merge([r,g,b])
        img_data_list.append(img)

Loading images from no folder

Loading images from yes folder



In [6]:
num_classes = len(classes_names_list)
print(num_classes)

2


In [7]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [8]:
print (img_data.shape)

(253, 224, 224, 3)


In [9]:
#show one training sample
from matplotlib import pyplot as plt
plt.imshow(img_data[97])
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [11]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:98]=0
classes[98:]=1

In [12]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

Using TensorFlow backend.


In [13]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=456)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=456)

In [15]:
# Check the number of images in each dataset split
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(202, 224, 224, 3) (51, 224, 224, 3)
(202, 2) (51, 2)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [17]:
#### Build the model
model = Sequential()

model.add(Conv2D(32, (3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 64)       

In [20]:
%%time
hist = model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1, validation_data=(X_test, y_test))

Train on 202 samples, validate on 51 samples
Epoch 1/20
202/202 [==============================] - 19s 92ms/step - loss: 1.0266 - accuracy: 0.6188 - val_loss: 0.6716 - val_accuracy: 0.5980
Epoch 2/20
202/202 [==============================] - 20s 98ms/step - loss: 0.6060 - accuracy: 0.6386 - val_loss: 0.6428 - val_accuracy: 0.5784
Epoch 3/20
202/202 [==============================] - 19s 96ms/step - loss: 0.5724 - accuracy: 0.7054 - val_loss: 0.6145 - val_accuracy: 0.8235
Epoch 4/20
202/202 [==============================] - 19s 93ms/step - loss: 0.5283 - accuracy: 0.7277 - val_loss: 0.5978 - val_accuracy: 0.8039
Epoch 5/20
202/202 [==============================] - 20s 98ms/step - loss: 0.5237 - accuracy: 0.7500 - val_loss: 0.6082 - val_accuracy: 0.8235
Epoch 6/20
202/202 [==============================] - 20s 97ms/step - loss: 0.4813 - accuracy: 0.7921 - val_loss: 0.5721 - val_accuracy: 0.8137
Epoch 7/20
202/202 [==============================] - 20s 99ms/step - loss: 0.4590 - accura

In [21]:
score = model.evaluate(X_test, y_test, batch_size=32)

print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

51/51 [==============================] - 1s 25ms/step
Test Loss: 1.1283915241559346
Test Accuracy: 0.8725489974021912


In [22]:
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)

In [23]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

[1 1 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1
 1 0 0 1 1 0 0 0 1 1 1 1 1 1]


In [24]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

[[17  6]
 [ 1 27]]


## Data Augementation

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.1, 
    zoom_range=0.4, 
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

test_data_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_data_gen.flow_from_directory(
        DATA_PATH,
        target_size=(224, 224), 
        batch_size=32,
        class_mode='binary',
        color_mode='rgb', 
        shuffle=True,  
        save_to_dir='Train_Augmented_Images', 
        save_prefix='TrainAugmented', 
        save_format='jpeg')

test_generator = test_data_gen.flow_from_directory(
        DATA_PATH,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        color_mode='rgb',
        shuffle=True, 
        seed=None, 
        save_to_dir='Test_Augmented_Images', 
        save_prefix='TestAugmented', 
        save_format='jpeg')

Found 253 images belonging to 2 classes.
Found 253 images belonging to 2 classes.


In [26]:
train_generator.class_indices

{'no': 0, 'yes': 1}

In [27]:
test_generator.class_indices

{'no': 0, 'yes': 1}

In [28]:
#### Build the model
model = Sequential()

model.add(Conv2D(32, (3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 53, 53, 64)       

In [31]:
%%time
model.fit_generator(train_generator, epochs=20, validation_data=test_generator)

Epoch 1/20
8/8 [==============================] - 34s 4s/step - loss: 0.9577 - accuracy: 0.5929 - val_loss: 0.6105 - val_accuracy: 0.6126
Epoch 2/20
8/8 [==============================] - 31s 4s/step - loss: 0.6615 - accuracy: 0.6126 - val_loss: 0.6026 - val_accuracy: 0.6126
Epoch 3/20
8/8 [==============================] - 31s 4s/step - loss: 0.6616 - accuracy: 0.6206 - val_loss: 0.6585 - val_accuracy: 0.6601
Epoch 4/20
8/8 [==============================] - 31s 4s/step - loss: 0.6708 - accuracy: 0.6324 - val_loss: 0.6364 - val_accuracy: 0.6166
Epoch 5/20
8/8 [==============================] - 33s 4s/step - loss: 0.6733 - accuracy: 0.6126 - val_loss: 0.5941 - val_accuracy: 0.6166
Epoch 6/20
8/8 [==============================] - 34s 4s/step - loss: 0.6698 - accuracy: 0.6443 - val_loss: 0.6564 - val_accuracy: 0.6364
Epoch 7/20
8/8 [==============================] - 31s 4s/step - loss: 0.6567 - accuracy: 0.6245 - val_loss: 0.4824 - val_accuracy: 0.6126
Epoch 8/20
8/8 [==================

### TL - VGG16

In [32]:
from keras.layers import Input, Dense
from keras.models import Model

In [33]:
image_input = Input(shape=(224, 224, 3))

In [34]:
from keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=False, weights='imagenet')

In [35]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model = Model(image_input, out)

In [36]:
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [37]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False

In [38]:
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [39]:
custom_vgg_model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [40]:
%%time
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1, validation_data=(X_test, y_test))

Train on 202 samples, validate on 51 samples
Epoch 1/20
202/202 [==============================] - 51s 254ms/step - loss: 4.0484 - accuracy: 0.5693 - val_loss: 0.7872 - val_accuracy: 0.4510
Epoch 2/20
202/202 [==============================] - 55s 273ms/step - loss: 1.2319 - accuracy: 0.4901 - val_loss: 0.6770 - val_accuracy: 0.4314
Epoch 3/20
202/202 [==============================] - 52s 257ms/step - loss: 0.8291 - accuracy: 0.6089 - val_loss: 0.4768 - val_accuracy: 0.8039
Epoch 4/20
202/202 [==============================] - 53s 263ms/step - loss: 0.5067 - accuracy: 0.8069 - val_loss: 0.7264 - val_accuracy: 0.6667
Epoch 5/20
202/202 [==============================] - 52s 260ms/step - loss: 0.5261 - accuracy: 0.7327 - val_loss: 0.4677 - val_accuracy: 0.8627
Epoch 6/20
202/202 [==============================] - 51s 253ms/step - loss: 0.4548 - accuracy: 0.7970 - val_loss: 0.5919 - val_accuracy: 0.7647
Epoch 7/20
202/202 [==============================] - 52s 256ms/step - loss: 0.4371 -

In [41]:
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=32, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

51/51 [==============================] - 12s 239ms/step
[INFO] loss=0.2806, accuracy: 92.1569%


In [42]:
Y_train_pred = custom_vgg_model.predict(X_test)

In [43]:
y_train_pred = np.argmax(Y_train_pred, axis=1)
print(y_train_pred)

[1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1
 1 0 0 1 1 0 0 0 0 1 1 0 1 1]


In [44]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_train_pred))

[[19  4]
 [ 0 28]]


# Conclusion
Compare to all models VGG gives best output